# 먼저 라이브러리들을 사용해서 이진 분류 해보자

In [ ]:
import pandas as pd
import tensorflow as tf

In [ ]:
#데이터 불러오기
df = pd.read_csv('binary_dataset.csv')
#df

In [ ]:
df.head()

,x1,x2,x3,x4,x5,x6,x7,x8,y
0,136.093750,51.691005,-0.045909,-0.271816,9.342809,38.096400,4.345438,18.673649,0
1,99.367188,41.572202,1.547197,4.154106,27.555184,61.719016,2.208808,3.662680,1
2,100.890625,51.890394,0.627487,-0.026498,3.883779,23.045267,6.953168,52.279440,0
3,120.554688,45.549905,0.282924,0.419909,1.358696,13.079034,13.312141,212.597029,1
4,121.882812,53.042675,0.200521,-0.282219,2.116221,16.580876,8.947603,91.011762,0


In [ ]:
df.shape

(20, 9)

In [ ]:
features = df.drop(columns = 'y')
label = df.iloc[:,8]

In [ ]:
# 학습데이터는 0.8 테스트 데이터는 0.2
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(features, label, test_size=0.2, random_state=42)

In [ ]:
print(X_train.shape)
print(X_test.shape)

(16, 8)
(4, 8)


In [ ]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Flatten(input_shape=(8,)))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

In [ ]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train, epochs=1)

1/1 [==============================] - 2s 2s/step - loss: 21.2685 - accuracy: 0.6250


In [ ]:
model.evaluate(X_test,  y_test, verbose=2)

1/1 - 0s - loss: 39.4972 - accuracy: 0.5000 - 142ms/epoch - 142ms/step


[39.497222900390625, 0.5]

# 라이브러리를 안쓰는 프로젝트

In [1]:
#얘네만 사용하기

import numpy as np
import pandas as pd
import csv
import matplotlib.pyplot as plt

In [2]:
#데이터 불러오기
def load_data(url):

  df = pd.read_csv(url)
  return df

In [3]:
urls = 'binary_dataset.csv'
df = load_data(urls)
df.head()

,x1,x2,x3,x4,x5,x6,x7,x8,y
0,136.093750,51.691005,-0.045909,-0.271816,9.342809,38.096400,4.345438,18.673649,0
1,99.367188,41.572202,1.547197,4.154106,27.555184,61.719016,2.208808,3.662680,1
2,100.890625,51.890394,0.627487,-0.026498,3.883779,23.045267,6.953168,52.279440,0
3,120.554688,45.549905,0.282924,0.419909,1.358696,13.079034,13.312141,212.597029,1
4,121.882812,53.042675,0.200521,-0.282219,2.116221,16.580876,8.947603,91.011762,0


In [4]:
#학습데이터와 테스트 데이터를 0.8 0.2 로 나누기
#20개중에서 무작위로 4개를 뽑고 얘는 테스트 데이터로 삼자
#매개변수를 test_size = 0.2로 했을때 그것만 뽑으려면 (전체 데이터 개수 * test_size매개변수로 해서 그것만 뽑자)


def data_split(df,test_size):
  li = []
  start = 0
  end   = df.shape[0] 
  test_ = df.shape[0]*test_size

  for _ in range(int(test_)):
    a = np.random.randint(start,end, size=1)[0]
    while a in li:
      a = np.random.randint(start,end, size=1)[0]
    li.append(a)

  X_test,y_test   = df.iloc[li,:8],df.iloc[li,8]
  X_train,y_train = df.drop(index = li).iloc[:,:8],df.drop(index = li).iloc[:,8]

  return X_train,y_train,X_test,y_test




In [5]:
X_train,y_train,X_test,y_test = data_split(df,0.2)


In [6]:
X_test

,x1,x2,x3,x4,x5,x6,x7,x8
18,103.523438,45.725739,0.336533,0.520558,11.289298,39.116453,3.509139,11.503980
16,123.531250,53.348784,0.072078,-0.071601,0.781773,10.570833,17.118300,339.660826
11,114.281250,41.253965,0.411821,0.616996,2.412207,20.427942,9.198392,88.370580
8,83.679688,36.379281,0.572532,2.664611,4.040970,23.169129,7.006681,53.514005


In [8]:
#파라미터와 편향 생성하기
#가중치 초기화는 표준정규분포에서 뽑기
#원래 기울기 초기화의 경우 역전파에서 가중치 소실이나 폭발이 일어나서 xavier나 he를 쓰지만 여기서는 역전파가 없으니 그냥 초기화
#편향의 경우 0으로 초기화하거나 0.01로 초기화하는 경우가 많아서 0.01로 

def para(node_num):
  
  node_wei = np.random.randn(node_num)
  bias     = 0.01

  return node_wei,bias


In [9]:
node_wei,bias = para(8)


In [10]:
#데이터 셔플해서 인덱스만 다시 설정하는 메서드

def data_shu(df):
  df = df.sample(frac=1).reset_index(drop = True)
  return df

In [17]:
df_li = [X_train,X_test,y_train,y_test]
for i in df_li:
  data_shu(i)

In [21]:
#미니 배치로 데이터 나누기

def mini_batch(df):
  #4개씩 x_train을 나누기
  mini_1 = df.iloc[0:4,:]
  mini_2 = df.iloc[4:8,:]
  mini_3 = df.iloc[8:12,:]
  mini_4 = df.iloc[12:16,:]
  return mini_1,mini_2,mini_3,mini_4


In [24]:
mini_1,mini_2,mini_3,mini_4 = mini_batch(X_train)

In [25]:
mini_1

,x1,x2,x3,x4,x5,x6,x7,x8
0,136.093750,51.691005,-0.045909,-0.271816,9.342809,38.096400,4.345438,18.673649
1,99.367188,41.572202,1.547197,4.154106,27.555184,61.719016,2.208808,3.662680
2,100.890625,51.890394,0.627487,-0.026498,3.883779,23.045267,6.953168,52.279440
3,120.554688,45.549905,0.282924,0.419909,1.358696,13.079034,13.312141,212.597029


In [27]:
node_wei

array([ 0.32568431, -0.74935749,  0.43375019,  0.56893793,  0.18724268,
       -0.54340316, -0.25478629,  1.01192447])

In [29]:
mini_1 * node_wei

,x1,x2,x3,x4,x5,x6,x7,x8
0,44.323599,-38.735042,-0.019913,-0.154647,1.749373,-20.701704,-1.107158,18.896322
1,32.362334,-31.152441,0.671097,2.363428,5.159507,-33.538308,-0.562774,3.706356
2,32.858494,-38.884456,0.272172,-0.015076,0.727209,-12.522871,-1.771572,52.902845
3,39.262771,-34.133163,0.122718,0.238902,0.254406,-7.107189,-3.391751,215.132135


In [37]:
sum(mini_1.iloc[0,:]*node_wei)

4.250830656572992

In [38]:
def forward_propa(df,weight,bias):
  #각 데이터를 가중치를 곱하고 bias를 더해서 리턴하기
  return sum(df * weight) + bias


In [40]:
res = forward_propa(mini_1.iloc[0,:],node_wei,bias)

In [32]:
#가중치연산을 통해 나온 값을 활성화 함수 sigmoid를 씌우서 확률값 리턴하기
def sigmoid(x):
  return 1 / (1 + np.exp(-x))

In [41]:
sigmoid(res) #로지스틱 회귀모델은 0과 1 사이의 값을 출력하며 출력값이 0.5 이상일 경우는 Class1로 0.5 미만일 경우는 Class0으로 분류합니다.

0.9860857614301277

In [ ]:
def is_bibary(res):
  #sigmoid 활성화 함수로 나온 값이 0인지 1인지 예측하기